# Task description
- Classify the speakers of given features.
- Main goal: Learn how to use transformer.
- Baselines:
  - Easy: Run sample code and know how to use transformer.
  - Medium: Know how to adjust parameters of transformer.
  - Strong: Construct [conformer](https://arxiv.org/abs/2005.08100) which is a variety of transformer.
  - Boss: Implement [Self-Attention Pooling](https://arxiv.org/pdf/2008.01077v1.pdf) & [Additive Margin Softmax](https://arxiv.org/pdf/1801.05599.pdf) to further boost the performance.

- Other links
  - Kaggle: [link](https://www.kaggle.com/t/ac77388c90204a4c8daebeddd40ff916)
  - Slide: [link](https://docs.google.com/presentation/d/1HLAj7UUIjZOycDe7DaVLSwJfXVd3bXPOyzSb6Zk3hYU/edit?usp=sharing)
  - Data: [link](https://drive.google.com/drive/folders/1vI1kuLB-q1VilIftiwnPOCAeOOFfBZge?usp=sharing)

# Download dataset
- Data is [here](https://drive.google.com/drive/folders/1vI1kuLB-q1VilIftiwnPOCAeOOFfBZge?usp=sharing)

!wget https://github.com/MachineLearningHW/ML_HW4_Dataset/releases/latest/download/Dataset.tar.gz.partaa
!wget https://github.com/MachineLearningHW/ML_HW4_Dataset/releases/latest/download/Dataset.tar.gz.partab
!wget https://github.com/MachineLearningHW/ML_HW4_Dataset/releases/latest/download/Dataset.tar.gz.partac
!wget https://github.com/MachineLearningHW/ML_HW4_Dataset/releases/latest/download/Dataset.tar.gz.partad

!cat Dataset.tar.gz.part* > Dataset.tar.gz

# unzip the file
!tar zxvf Dataset.tar.gz

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Fix Random Seed

In [2]:
import numpy as np
import torch
import random

def set_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

set_seed(87)

In [3]:
print(f"Using: {'GPU' if torch.cuda.is_available() else 'CPU'}")

Using: GPU


# Data

## Dataset
- Original dataset is [Voxceleb2](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/vox2.html).
- The [license](https://creativecommons.org/licenses/by/4.0/) and [complete version](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/files/license.txt) of Voxceleb2.
- We randomly select 600 speakers from Voxceleb2.
- Then preprocess the raw waveforms into mel-spectrograms.

- Args:
  - data_dir: The path to the data directory.
  - metadata_path: The path to the metadata.
  - segment_len: The length of audio segment for training.
- The architecture of data directory \\
  - data directory \\
  |---- metadata.json \\
  |---- testdata.json \\
  |---- mapping.json \\
  |---- uttr-{random string}.pt \\

- The information in metadata
  - "n_mels": The dimention of mel-spectrogram.
  - "speakers": A dictionary.
    - Key: speaker ids.
    - value: "feature_path" and "mel_len"


For efficiency, we segment the mel-spectrograms into segments in the traing step.

In [4]:
import os
import json
import torch
import random
from pathlib import Path
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence

class myDataset(Dataset):
	def __init__(self, data_dir, segment_len=192, use_spec_aug=True):
		self.data_dir = data_dir
		self.segment_len = segment_len
		self.use_spec_aug = use_spec_aug

		# Load the mapping from speaker neme to their corresponding id.
		mapping_path = Path(data_dir) / "mapping.json"
		mapping = json.load(mapping_path.open())
		self.speaker2id = mapping["speaker2id"]

		# Load metadata of training data.
		metadata_path = Path(data_dir) / "metadata.json"
		metadata = json.load(open(metadata_path))["speakers"]

		# Get the total number of speaker.
		self.speaker_num = len(metadata.keys())
		self.data = []
		for speaker in metadata.keys():
			for utterances in metadata[speaker]:
				self.data.append([utterances["feature_path"], self.speaker2id[speaker]])

	def __len__(self):
		return len(self.data)  # <-- Correctly indented

	# This method is now correctly indented to match __init__ and __len__
	def spec_augment(self, mel_spectrogram,
					time_mask_num=2, time_mask_width=10,
					freq_mask_num=2, freq_mask_width=8):
		"""新增SpecAug函数"""
		time_steps, n_mels = mel_spectrogram.shape
		augmented_mel = mel_spectrogram.clone()

		# Time Masking
		for _ in range(time_mask_num):
			t = random.randint(0, min(time_mask_width, time_steps))
			t0 = random.randint(0, max(1, time_steps - t))
			augmented_mel[t0:t0+t, :] = -20  # mel的padding值

		# Frequency Masking
		for _ in range(freq_mask_num):
			f = random.randint(0, min(freq_mask_width, n_mels))
			f0 = random.randint(0, max(1, n_mels - f))
			augmented_mel[:, f0:f0+f] = -20

		return augmented_mel

	def __getitem__(self, index):
		feat_path, speaker = self.data[index]
		# Load preprocessed mel-spectrogram.
		mel = torch.load(os.path.join(self.data_dir, feat_path))

		# Segmemt mel-spectrogram into "segment_len" frames.
		if len(mel) > self.segment_len:
			# Randomly get the starting point of the segment.
			start = random.randint(0, len(mel) - self.segment_len)
			# Get a segment with "segment_len" frames.
			mel = torch.FloatTensor(mel[start:start+self.segment_len])
		else:
			mel = torch.FloatTensor(mel)

		# 新增：SpecAug（只在训练时使用）
		if self.use_spec_aug and hasattr(self, 'training') and self.training:
			mel = self.spec_augment(mel)

		# Turn the speaker id into long for computing loss later.
		speaker = torch.FloatTensor([speaker]).long()
		return mel, speaker

	def get_speaker_number(self):
		return self.speaker_num

## Dataloader
- Split dataset into training dataset(90%) and validation dataset(10%).
- Create dataloader to iterate the data.

In [5]:
import torch
from torch.utils.data import DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence


def collate_batch(batch):
	# Process features within a batch.
	"""Collate a batch of data."""
	mel, speaker = zip(*batch)
	# Because we train the model batch by batch, we need to pad the features in the same batch to make their lengths the same.
	mel = pad_sequence(mel, batch_first=True, padding_value=-20)    # pad log 10^(-20) which is very small value.
	# mel: (batch size, length, 40)
	return mel, torch.FloatTensor(speaker).long()


def get_dataloader(data_dir, batch_size, n_workers, use_spec_aug): # <-- Added use_spec_aug here
	"""Generate dataloader"""
	# All lines below are now correctly and consistently indented
	dataset = myDataset(data_dir, use_spec_aug=use_spec_aug)
	speaker_num = dataset.get_speaker_number()
	# Split dataset into training dataset and validation dataset
	trainlen = int(0.9 * len(dataset))
	lengths = [trainlen, len(dataset) - trainlen]
	trainset, validset = random_split(dataset, lengths)

	# Set the 'training' attribute for SpecAugment
	# This is a good practice to ensure Augmentation is only applied to the training set
	trainset.dataset.training = True
	validset.dataset.training = False

	train_loader = DataLoader(
		trainset,
		batch_size=batch_size,
		shuffle=True,
		drop_last=True,
		num_workers=n_workers,
		pin_memory=True,
		collate_fn=collate_batch,
	)
	valid_loader = DataLoader(
		validset,
		batch_size=batch_size,
		num_workers=n_workers,
		drop_last=True,
		pin_memory=True,
		collate_fn=collate_batch,
	)

	return train_loader, valid_loader, speaker_num

# Model
- TransformerEncoderLayer:
  - Base transformer encoder layer in [Attention Is All You Need](https://arxiv.org/abs/1706.03762)
  - Parameters:
    - d_model: the number of expected features of the input (required).

    - nhead: the number of heads of the multiheadattention models (required).

    - dim_feedforward: the dimension of the feedforward network model (default=2048).

    - dropout: the dropout value (default=0.1).

    - activation: the activation function of intermediate layer, relu or gelu (default=relu).

- TransformerEncoder:
  - TransformerEncoder is a stack of N transformer encoder layers
  - Parameters:
    - encoder_layer: an instance of the TransformerEncoderLayer() class (required).

    - num_layers: the number of sub-encoder-layers in the encoder (required).

    - norm: the layer normalization component (optional).

In [6]:
# https://github.com/pytorch/audio/blob/main/src/torchaudio/models/conformer.py
# https://github.com/yuting-NTU/NTU-ML2022-Spring/blob/main/r10922196_hw4/hw04.ipynb
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import Optional, Tuple
import random

# ==================== 1. SpecAug 数据增强 ====================
def spec_augment(mel_spectrogram, time_mask_num=2, time_mask_width=10,
                freq_mask_num=2, freq_mask_width=8):
    """SpecAug数据增强"""
    time_steps, n_mels = mel_spectrogram.shape
    augmented_mel = mel_spectrogram.clone()

    # Time Masking
    for _ in range(time_mask_num):
        t = random.randint(0, min(time_mask_width, time_steps))
        t0 = random.randint(0, max(1, time_steps - t))
        augmented_mel[t0:t0+t, :] = -20

    # Frequency Masking
    for _ in range(freq_mask_num):
        f = random.randint(0, min(freq_mask_width, n_mels))
        f0 = random.randint(0, max(1, n_mels - f))
        augmented_mel[:, f0:f0+f] = -20

    return augmented_mel

# ==================== 2. 新的完整Conformer实现 ====================
def _lengths_to_padding_mask(lengths: torch.Tensor) -> torch.Tensor:
    batch_size = lengths.shape[0]
    max_length = int(torch.max(lengths).item())
    padding_mask = torch.arange(max_length, device=lengths.device, dtype=lengths.dtype).expand(
        batch_size, max_length
    ) >= lengths.unsqueeze(1)
    return padding_mask

class _ConvolutionModule(torch.nn.Module):
    def __init__(self, input_dim: int, num_channels: int, depthwise_kernel_size: int,
                 dropout: float = 0.0, bias: bool = False, use_group_norm: bool = False) -> None:
        super().__init__()
        if (depthwise_kernel_size - 1) % 2 != 0:
            raise ValueError("depthwise_kernel_size must be odd to achieve 'SAME' padding.")
        self.layer_norm = torch.nn.LayerNorm(input_dim)
        self.sequential = torch.nn.Sequential(
            torch.nn.Conv1d(input_dim, 2 * num_channels, 1, stride=1, padding=0, bias=bias),
            torch.nn.GLU(dim=1),
            torch.nn.Conv1d(num_channels, num_channels, depthwise_kernel_size, stride=1,
                          padding=(depthwise_kernel_size - 1) // 2, groups=num_channels, bias=bias),
            torch.nn.GroupNorm(num_groups=1, num_channels=num_channels) if use_group_norm
            else torch.nn.BatchNorm1d(num_channels),
            torch.nn.SiLU(),
            torch.nn.Conv1d(num_channels, input_dim, kernel_size=1, stride=1, padding=0, bias=bias),
            torch.nn.Dropout(dropout),
        )

    def forward(self, input: torch.Tensor) -> torch.Tensor:
        x = self.layer_norm(input)
        x = x.transpose(1, 2)
        x = self.sequential(x)
        return x.transpose(1, 2)

class _FeedForwardModule(torch.nn.Module):
    def __init__(self, input_dim: int, hidden_dim: int, dropout: float = 0.0) -> None:
        super().__init__()
        self.sequential = torch.nn.Sequential(
            torch.nn.LayerNorm(input_dim),
            torch.nn.Linear(input_dim, hidden_dim, bias=True),
            torch.nn.SiLU(),
            torch.nn.Dropout(dropout),
            torch.nn.Linear(hidden_dim, input_dim, bias=True),
            torch.nn.Dropout(dropout),
        )

    def forward(self, input: torch.Tensor) -> torch.Tensor:
        return self.sequential(input)

class ConformerLayer(torch.nn.Module):
    def __init__(self, input_dim: int, ffn_dim: int, num_attention_heads: int,
                 depthwise_conv_kernel_size: int, dropout: float = 0.0,
                 use_group_norm: bool = False, convolution_first: bool = False) -> None:
        super().__init__()

        self.ffn1 = _FeedForwardModule(input_dim, ffn_dim, dropout=dropout)
        self.self_attn_layer_norm = torch.nn.LayerNorm(input_dim)
        self.self_attn = torch.nn.MultiheadAttention(input_dim, num_attention_heads, dropout=dropout)
        self.self_attn_dropout = torch.nn.Dropout(dropout)
        self.conv_module = _ConvolutionModule(
            input_dim=input_dim, num_channels=input_dim,
            depthwise_kernel_size=depthwise_conv_kernel_size,
            dropout=dropout, bias=True, use_group_norm=use_group_norm)
        self.ffn2 = _FeedForwardModule(input_dim, ffn_dim, dropout=dropout)
        self.final_layer_norm = torch.nn.LayerNorm(input_dim)
        self.convolution_first = convolution_first

    def _apply_convolution(self, input: torch.Tensor) -> torch.Tensor:
        residual = input
        input = input.transpose(0, 1)
        input = self.conv_module(input)
        input = input.transpose(0, 1)
        input = residual + input
        return input

    def forward(self, input: torch.Tensor, key_padding_mask: Optional[torch.Tensor]) -> torch.Tensor:
        residual = input
        x = self.ffn1(input)
        x = x * 0.5 + residual

        if self.convolution_first:
            x = self._apply_convolution(x)

        residual = x
        x = self.self_attn_layer_norm(x)
        x, _ = self.self_attn(query=x, key=x, value=x, key_padding_mask=key_padding_mask, need_weights=False)
        x = self.self_attn_dropout(x)
        x = x + residual

        if not self.convolution_first:
            x = self._apply_convolution(x)

        residual = x
        x = self.ffn2(x)
        x = x * 0.5 + residual
        x = self.final_layer_norm(x)
        return x

class Conformer(torch.nn.Module):
    def __init__(self, input_dim: int, num_heads: int, ffn_dim: int, num_layers: int,
                 depthwise_conv_kernel_size: int, dropout: float = 0.0,
                 use_group_norm: bool = False, convolution_first: bool = False):
        super().__init__()
        self.conformer_layers = torch.nn.ModuleList([
            ConformerLayer(input_dim, ffn_dim, num_heads, depthwise_conv_kernel_size,
                         dropout=dropout, use_group_norm=use_group_norm,
                         convolution_first=convolution_first)
            for _ in range(num_layers)
        ])

    def forward(self, input: torch.Tensor, lengths: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        encoder_padding_mask = _lengths_to_padding_mask(lengths)
        x = input.transpose(0, 1)
        for layer in self.conformer_layers:
            x = layer(x, encoder_padding_mask)
        return x.transpose(0, 1), lengths

# ==================== 3. Self-Attention Pooling ====================
class SelfAttentionPooling(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.W = nn.Linear(input_dim, 1)

    def forward(self, batch_rep):
        att_w = F.softmax(self.W(batch_rep).squeeze(-1), dim=-1).unsqueeze(-1)
        utter_rep = torch.sum(batch_rep * att_w, dim=1)
        return utter_rep

# ==================== 4. 主分类器 ====================
class Classifier(nn.Module):
    def __init__(self, d_model=160, n_spks=600, dropout=0.1,
                 n_layers=2, ffn_dim=320, n_heads=4, device="cuda",
                 use_group_norm=False, convolution_first=False, use_sap=True):
        super().__init__()

        # 特征升维层
        self.prenet = nn.Linear(40, d_model)

        # 完整的Conformer编码器
        self.conformer = Conformer(
            input_dim=d_model,
            num_heads=n_heads,
            ffn_dim=ffn_dim,              # 通常设为2-4倍d_model
            num_layers=n_layers,
            depthwise_conv_kernel_size=31, # 使用标准值
            dropout=dropout,
            use_group_norm=use_group_norm, # 可以实验GroupNorm
            convolution_first=convolution_first,  # 可以实验卷积在前
        )

        # 池化策略选择
        self.use_sap = use_sap
        if use_sap:
            self.pooling = SelfAttentionPooling(d_model)
        else:
            self.pooling = lambda x: x.mean(dim=1)  # 简单mean pooling

        # 改进的预测层（参考Boss baseline2但简化）
        self.pred_layer = nn.Sequential(
            nn.BatchNorm1d(d_model),
            nn.Linear(d_model, d_model),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_model, n_spks),
        )

        self.device = device

    def forward(self, mels):
        # 1. 特征升维: (batch, time, 40) -> (batch, time, d_model)
        out = self.prenet(mels)

        # 2. Conformer编码: 需要长度信息
        lengths = torch.full((out.shape[0],), out.shape[1], dtype=torch.long).to(self.device)
        out, _ = self.conformer(out, lengths)  # (batch, time, d_model)

        # 3. 池化: (batch, time, d_model) -> (batch, d_model)
        if self.use_sap:
            stats = self.pooling(out)
        else:
            stats = out.mean(dim=1)

        # 4. 最终分类: (batch, d_model) -> (batch, n_spks)
        out = self.pred_layer(stats)

        return out

# ==================== 5. 修改Dataset添加SpecAug ====================
class myDataset(Dataset):
    def __init__(self, data_dir, segment_len=192, use_spec_aug=True):
        self.data_dir = data_dir
        self.segment_len = segment_len
        self.use_spec_aug = use_spec_aug
        self.training = True  # 训练标志

        # 原来的初始化代码...
        mapping_path = Path(data_dir) / "mapping.json"
        mapping = json.load(mapping_path.open())
        self.speaker2id = mapping["speaker2id"]

        metadata_path = Path(data_dir) / "metadata.json"
        metadata = json.load(open(metadata_path))["speakers"]

        self.speaker_num = len(metadata.keys())
        self.data = []
        for speaker in metadata.keys():
            for utterances in metadata[speaker]:
                self.data.append([utterances["feature_path"], self.speaker2id[speaker]])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        feat_path, speaker = self.data[index]
        mel = torch.load(os.path.join(self.data_dir, feat_path))

        # 随机分段
        if len(mel) > self.segment_len:
            start = random.randint(0, len(mel) - self.segment_len)
            mel = torch.FloatTensor(mel[start:start+self.segment_len])
        else:
            mel = torch.FloatTensor(mel)

        # SpecAug数据增强（只在训练时）
        if self.use_spec_aug and self.training:
            mel = spec_augment(mel)

        speaker = torch.FloatTensor([speaker]).long()
        return mel, speaker

    def get_speaker_number(self):
        return self.speaker_num

# Learning rate schedule
- For transformer architecture, the design of learning rate schedule is different from that of CNN.
- Previous works show that the warmup of learning rate is useful for training models with transformer architectures.
- The warmup schedule
  - Set learning rate to 0 in the beginning.
  - The learning rate increases linearly from 0 to initial learning rate during warmup period.

In [7]:
import math

import torch
from torch.optim import Optimizer
from torch.optim.lr_scheduler import LambdaLR


def get_cosine_schedule_with_warmup(
	optimizer: Optimizer,
	num_warmup_steps: int,
	num_training_steps: int,
	num_cycles: float = 0.5,
	last_epoch: int = -1,
):
	"""
	Create a schedule with a learning rate that decreases following the values of the cosine function between the
	initial lr set in the optimizer to 0, after a warmup period during which it increases linearly between 0 and the
	initial lr set in the optimizer.

	Args:
		optimizer (:class:`~torch.optim.Optimizer`):
		The optimizer for which to schedule the learning rate.
		num_warmup_steps (:obj:`int`):
		The number of steps for the warmup phase.
		num_training_steps (:obj:`int`):
		The total number of training steps.
		num_cycles (:obj:`float`, `optional`, defaults to 0.5):
		The number of waves in the cosine schedule (the defaults is to just decrease from the max value to 0
		following a half-cosine).
		last_epoch (:obj:`int`, `optional`, defaults to -1):
		The index of the last epoch when resuming training.

	Return:
		:obj:`torch.optim.lr_scheduler.LambdaLR` with the appropriate schedule.
	"""
	def lr_lambda(current_step):
		# Warmup
		if current_step < num_warmup_steps:
			return float(current_step) / float(max(1, num_warmup_steps))
		# decadence
		progress = float(current_step - num_warmup_steps) / float(
			max(1, num_training_steps - num_warmup_steps)
		)
		return max(
			0.0, 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress))
		)

	return LambdaLR(optimizer, lr_lambda, last_epoch)

# Model Function
- Model forward function.

In [8]:
import torch


def model_fn(batch, model, criterion, device):
	"""Forward a batch through the model."""

	mels, labels = batch
	mels = mels.to(device)
	labels = labels.to(device)

	outs = model(mels)

	loss = criterion(outs, labels)

	# Get the speaker id with highest probability.
	preds = outs.argmax(1)
	# Compute accuracy.
	accuracy = torch.mean((preds == labels).float())

	return loss, accuracy

# Validate
- Calculate accuracy of the validation set.

In [9]:
from tqdm import tqdm
import torch


def valid(dataloader, model, criterion, device):
	"""Validate on validation set."""

	model.eval()
	running_loss = 0.0
	running_accuracy = 0.0
	pbar = tqdm(total=len(dataloader.dataset), ncols=0, desc="Valid", unit=" uttr")

	for i, batch in enumerate(dataloader):
		with torch.no_grad():
			loss, accuracy = model_fn(batch, model, criterion, device)
			running_loss += loss.item()
			running_accuracy += accuracy.item()

		pbar.update(dataloader.batch_size)
		pbar.set_postfix(
			loss=f"{running_loss / (i+1):.2f}",
			accuracy=f"{running_accuracy / (i+1):.2f}",
		)

	pbar.close()
	model.train()

	return running_accuracy / len(dataloader)

# Main function

In [10]:
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader, random_split


def parse_args():
	"""arguments"""
	config = {
		# "data_dir": "./Dataset",
		"data_dir": "/content/drive/MyDrive/Dataset",
		"save_path": "/content/drive/MyDrive/model.ckpt",
		"batch_size": 32,
		"n_workers": 8,
		"valid_steps": 2000,
        #学习率 = (当前步数 / warmup_steps) × 目标学习率
		"warmup_steps": 2000,  # 原版1000 -> 2000 (更平滑的学习率预热)
		"save_steps": 10000,
		"total_steps": 200000, #原70000

         # 新的模型参数
        "d_model": 160,
        "n_layers": 2,             # 先用2层
        "ffn_dim": 320,           # 2倍d_model
        "n_heads": 4,
        "dropout": 0.1,
        "use_group_norm": False,   # 可实验True
        "convolution_first": False,# 可实验True
        "use_sap": True,          # 使用SAP
        "use_spec_aug": True,     # 使用SpecAug
	}

	return config


def main(data_dir, save_path, batch_size, n_workers, valid_steps, warmup_steps, total_steps, save_steps,
         d_model, n_layers, ffn_dim, n_heads, dropout, use_group_norm, convolution_first, use_sap, use_spec_aug, early_stop=15):

	"""Main function."""
	device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
	print(f"[Info]: Use {device} now!")

	train_loader, valid_loader, speaker_num = get_dataloader(data_dir, batch_size, n_workers, use_spec_aug)
	train_iterator = iter(train_loader)
	print(f"[Info]: Finish loading data!",flush = True)

	model = Classifier(
		d_model=d_model,
		n_spks=speaker_num,
		dropout=dropout,
		n_layers=n_layers,
		ffn_dim=ffn_dim,
		n_heads=n_heads,
		device=device,
		use_group_norm=use_group_norm,
		convolution_first=convolution_first,
		use_sap=use_sap,).to(device)

	criterion = nn.CrossEntropyLoss()
	optimizer = AdamW(model.parameters(), lr=1e-3)
	scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, total_steps)
	print(f"[Info]: Finish creating model!",flush = True)

	best_accuracy = -1.0
	best_state_dict = None
	early_stop_count = 0 #早停机制

	pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

	for step in range(total_steps):
		try:
			batch = next(train_iterator)
		except StopIteration:
			train_iterator = iter(train_loader)
			batch = next(train_iterator)

		loss, accuracy = model_fn(batch, model, criterion, device)
		batch_loss = loss.item()
		batch_accuracy = accuracy.item()

		loss.backward()
		optimizer.step()
		scheduler.step()
		optimizer.zero_grad()

		pbar.update()
		pbar.set_postfix(
			loss=f"{batch_loss:.2f}",
			accuracy=f"{batch_accuracy:.2f}",
			step=step + 1,
		)

		if (step + 1) % valid_steps == 0:
			pbar.close()
			valid_accuracy = valid(valid_loader, model, criterion, device)

			if valid_accuracy > best_accuracy:
				best_accuracy = valid_accuracy
				best_state_dict = model.state_dict()
				early_stop_count = 0
			else:
				early_stop_count += 1

			if early_stop_count >= early_stop:
				print(f"Early stopping at step {step+1}")
				break

			pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

		if (step + 1) % save_steps == 0 and best_state_dict is not None:
			torch.save(best_state_dict, save_path)
			pbar.write(f"Step {step + 1}, best model saved. (accuracy={best_accuracy:.4f})")

	pbar.close()


if __name__ == "__main__":
	main(**parse_args())

[Info]: Use cuda now!


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


[Info]: Finish loading data!
[Info]: Finish creating model!


Train:  80% 1593/2000 [55:35<09:52,  1.46s/ step, accuracy=0.31, loss=3.76, step=1593]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Train: 100% 2000/2000 [56:17<00:00,  1.69s/ step, accuracy=0.22, loss=3.64, step=2000]
Valid: 100% 5664/5667 [06:29<00:00, 14.54 uttr/s, accuracy=0.31, loss=3.11]
Train: 100% 2000/2000 [03:22<00:00,  9.87 step/s, accuracy=0.47, loss=2.09, step=4000]
Valid: 100% 5664/5667 [00:18<00:00, 303.08 uttr/s, accuracy=0.51, loss=2.09]
Train: 100% 2000/2000 [03:19<00:00, 10.04 step/s, accuracy=0.69, loss=1.42, step=6000]
Valid: 100% 5664/5667 [00:17<00:00, 321.88 ut

Step 10000, best model saved. (accuracy=0.7223)


...

Step 180000, best model saved. (accuracy=0.9366)


Train: 100% 2000/2000 [03:27<00:00,  9.62 step/s, accuracy=0.97, loss=0.08, step=182000]
Valid: 100% 5664/5667 [00:19<00:00, 295.91 uttr/s, accuracy=0.94, loss=0.32]
Train: 100% 2000/2000 [03:31<00:00,  9.47 step/s, accuracy=1.00, loss=0.03, step=184000]
Valid: 100% 5664/5667 [00:19<00:00, 294.72 uttr/s, accuracy=0.94, loss=0.34]
Train: 100% 2000/2000 [03:32<00:00,  9.43 step/s, accuracy=1.00, loss=0.00, step=186000]
Valid: 100% 5664/5667 [00:18<00:00, 300.23 uttr/s, accuracy=0.94, loss=0.34]
Train: 100% 2000/2000 [03:30<00:00,  9.49 step/s, accuracy=1.00, loss=0.00, step=188000]
Valid: 100% 5664/5667 [00:19<00:00, 297.36 uttr/s, accuracy=0.94, loss=0.34]
Train: 100% 2000/2000 [03:25<00:00,  9.71 step/s, accuracy=1.00, loss=0.03, step=190000]
Valid: 100% 5664/5667 [00:18<00:00, 299.42 uttr/s, accuracy=0.94, loss=0.34]
Train:   0% 5/2000 [00:00<02:16, 14.61 step/s, accuracy=1.00, loss=0.01, step=190005]

Step 190000, best model saved. (accuracy=0.9386)


Train: 100% 2000/2000 [03:29<00:00,  9.56 step/s, accuracy=1.00, loss=0.01, step=192000]
Valid: 100% 5664/5667 [00:19<00:00, 289.84 uttr/s, accuracy=0.94, loss=0.34]
Train: 100% 2000/2000 [03:29<00:00,  9.53 step/s, accuracy=1.00, loss=0.01, step=194000]
Valid: 100% 5664/5667 [00:19<00:00, 290.16 uttr/s, accuracy=0.94, loss=0.35]
Train: 100% 2000/2000 [03:31<00:00,  9.48 step/s, accuracy=0.94, loss=0.09, step=196000]
Valid: 100% 5664/5667 [00:19<00:00, 292.98 uttr/s, accuracy=0.93, loss=0.34]
Train: 100% 2000/2000 [03:26<00:00,  9.70 step/s, accuracy=0.97, loss=0.09, step=198000]
Valid: 100% 5664/5667 [00:18<00:00, 300.66 uttr/s, accuracy=0.93, loss=0.33]
Train: 100% 2000/2000 [03:32<00:00,  9.43 step/s, accuracy=1.00, loss=0.01, step=2e+5]
Valid: 100% 5664/5667 [00:20<00:00, 281.13 uttr/s, accuracy=0.94, loss=0.34]
Train:   0% 0/2000 [00:00<?, ? step/s]


Step 200000, best model saved. (accuracy=0.9405)


# Inference

## Dataset of inference

In [11]:
import os
import json
import torch
from pathlib import Path
from torch.utils.data import Dataset


class InferenceDataset(Dataset):
	def __init__(self, data_dir):
		testdata_path = Path(data_dir) / "testdata.json"
		metadata = json.load(testdata_path.open())
		self.data_dir = data_dir
		self.data = metadata["utterances"]

	def __len__(self):
		return len(self.data)

	def __getitem__(self, index):
		utterance = self.data[index]
		feat_path = utterance["feature_path"]
		mel = torch.load(os.path.join(self.data_dir, feat_path))

		return feat_path, mel


def inference_collate_batch(batch):
	"""Collate a batch of data."""
	feat_paths, mels = zip(*batch)

	return feat_paths, torch.stack(mels)

## Main funcrion of Inference

In [12]:
import json
import csv
from pathlib import Path
from tqdm.notebook import tqdm

import torch
from torch.utils.data import DataLoader

def parse_args():
    """inference arguments"""
    config = {
        "data_dir": "/content/drive/MyDrive/Dataset",
        "model_path": "/content/drive/MyDrive/model.ckpt",
        "output_path": "/content/drive/MyDrive/output.csv",

        # 新增：模型参数（要和训练时完全一致！）
        "d_model": 160,
        "n_layers": 2,
        "ffn_dim": 320,
        "n_heads": 4,
        "dropout": 0.1,
        "use_group_norm": False,
        "convolution_first": False,
        "use_sap": True,
    }
    return config

def main(data_dir, model_path, output_path,
         d_model, n_layers, ffn_dim, n_heads, dropout,
         use_group_norm, convolution_first, use_sap):
    """Main function."""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"[Info]: Use {device} now!")

    mapping_path = Path(data_dir) / "mapping.json"
    mapping = json.load(mapping_path.open())

    # 推理时的Dataset不需要SpecAug
    dataset = InferenceDataset(data_dir)
    dataloader = DataLoader(
        dataset,
        batch_size=1,
        shuffle=False,
        drop_last=False,
        num_workers=8,
        collate_fn=inference_collate_batch,
    )
    print(f"[Info]: Finish loading data!",flush = True)

    speaker_num = len(mapping["id2speaker"])

    # 关键修改：使用和训练时相同的参数创建模型
    model = Classifier(
        d_model=d_model,
        n_spks=speaker_num,
        dropout=dropout,
        n_layers=n_layers,
        ffn_dim=ffn_dim,
        n_heads=n_heads,
        device=device,
        use_group_norm=use_group_norm,
        convolution_first=convolution_first,
        use_sap=use_sap,
    ).to(device)

    model.load_state_dict(torch.load(model_path))
    model.eval()
    print(f"[Info]: Finish creating model!",flush = True)

    # 推理循环保持不变
    results = [["Id", "Category"]]
    for feat_paths, mels in tqdm(dataloader):
        with torch.no_grad():
            mels = mels.to(device)
            outs = model(mels)
            preds = outs.argmax(1).cpu().numpy()
            for feat_path, pred in zip(feat_paths, preds):
                results.append([feat_path, mapping["id2speaker"][str(pred)]])

    with open(output_path, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(results)

if __name__ == "__main__":
    main(**parse_args())

[Info]: Use cuda now!
[Info]: Finish loading data!
[Info]: Finish creating model!


  0%|          | 0/8000 [00:00<?, ?it/s]